In [1]:
#This notebook will contain calculations and analysis for the travel time of an average subway rider vs. an handicapped rider

In [2]:
#First I have two lists of random NYC addresses
import pandas as pd
import numpy as np

A_addresses = pd.read_csv('NYC_addresses_1.csv')
B_addresses = pd.read_csv('NYC_addresses_2.csv')

In [4]:
A_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"133-20 85 ST NEW YORK, NY 11417",133-20,85 ST,NEW YORK,NY,11417,-73.851338,40.673495,36,81,5800,3
1,"292 MONTAUK AVE NEW YORK, NY 11208",292,MONTAUK AVE,NEW YORK,NY,11208,-73.877478,40.671479,36,47,119400,3
2,"6 ROMER RD NEW YORK, NY 10304",6,ROMER RD,NEW YORK,NY,10304,-74.103098,40.593125,36,85,17701,3
3,"164-10 104 ST NEW YORK, NY 11414",164-10,104 ST,NEW YORK,NY,11414,-73.827662,40.651995,36,81,88400,6
4,"1429 SHORE PKWY NEW YORK, NY 11214",1429,SHORE PKWY,NEW YORK,NY,11214,-74.005699,40.598670,36,47,28200,1


In [5]:
B_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"3236 BOSTON RD NEW YORK, NY 10469",3236,BOSTON RD,NEW YORK,NY,10469,-73.854792,40.872162,36,5,34800,1
1,"970 AMSTERDAM AVE NEW YORK, NY 10025",970,AMSTERDAM AVE,NEW YORK,NY,10025,-73.965107,40.801581,36,61,19500,4
2,"1623 DECATUR ST NEW YORK, NY 11385",1623,DECATUR ST,NEW YORK,NY,11385,-73.901360,40.694898,36,81,55900,1
3,"12-06 151 PL NEW YORK, NY 11357",12-06,151 PL,NEW YORK,NY,11357,-73.811475,40.791270,36,81,98700,2
4,"135 BCH 56 PL NEW YORK, NY 11692",135,BCH 56 PL,NEW YORK,NY,11692,-73.787031,40.590884,36,81,97202,1


In [73]:
origins = A_addresses['address_full'].tolist()
origins

['133-20 85 ST NEW YORK, NY 11417 ',
 '292 MONTAUK AVE NEW YORK, NY 11208 ',
 '6 ROMER RD NEW YORK, NY 10304 ',
 '164-10 104 ST NEW YORK, NY 11414 ',
 '1429 SHORE PKWY NEW YORK, NY 11214 ',
 '196 22 ST NEW YORK, NY 11232 ',
 '715 SAGAMORE ST NEW YORK, NY 10462 ',
 '4730 RICHARDSON AVE NEW YORK, NY 10470 ',
 '94-17 84 ST NEW YORK, NY 11416 ',
 '2732 MILL AVE NEW YORK, NY 11234 ']

In [74]:
destinations = B_addresses['address_full'].tolist()
destinations

['3236 BOSTON RD NEW YORK, NY 10469 ',
 '970 AMSTERDAM AVE NEW YORK, NY 10025 ',
 '1623 DECATUR ST NEW YORK, NY 11385 ',
 '12-06 151 PL NEW YORK, NY 11357 ',
 '135 BCH 56 PL NEW YORK, NY 11692 ',
 '196 ROSEDALE AVE NEW YORK, NY 10312 ',
 '519 MADISON AVE NEW YORK, NY 10022 ',
 '197-26 CARPENTER AVE NEW YORK, NY 11423 ',
 '63-62 77 ST NEW YORK, NY 11379 ',
 '120-36 143 ST NEW YORK, NY 11436 ']

In [94]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('api_key')

In [183]:
#Run a test of the API
import requests

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': api_key,
    'X-Goog-FieldMask': 'routes.legs.steps.transitDetails,routes.localizedValues'}

json_data = {
    'origin': {
        'address': '853 Riverside Drive, New York, NY',
    },
    'destination': {
        'address': '75 9th Avenue, New York, NY',
    },
    'travelMode': 'TRANSIT',
    'computeAlternativeRoutes': True,
    'transitPreferences': {
        'allowedTravelModes': [
            'SUBWAY',
            'BUS',
        ],
    },
    'languageCode': 'en-US',
    'units': 'METRIC',
}

response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, json=json_data)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{\n  "origin":{\n    "address": "853 Riverside Drive, New York, NY"\n      },\n  "destination":{\n    "address": "75 9th Avenue, New York, NY" \n      },\n  "travelMode": "TRANSIT",\n  "computeAlternativeRoutes": true,\n  "transitPreferences": {\n    "allowedTravelModes": ["SUBWAY","BUS"]\n  },\n  "languageCode": "en-US",\n  "units": "METRIC"\n }'
#response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, data=data)

In [184]:
data =response.json()

In [185]:
data

{'routes': [{'legs': [{'steps': [{},
      {},
      {},
      {},
      {'transitDetails': {'stopDetails': {'arrivalStop': {'name': '59 St-Columbus Circle',
          'location': {'latLng': {'latitude': 40.768246,
            'longitude': -73.981926}}},
         'arrivalTime': '2025-07-29T19:57:55Z',
         'departureStop': {'name': '157 St',
          'location': {'latLng': {'latitude': 40.834042,
            'longitude': -73.944893}}},
         'departureTime': '2025-07-29T19:38:55Z'},
        'localizedValues': {'arrivalTime': {'time': {'text': '3:57\u202fPM'},
          'timeZone': 'America/New_York'},
         'departureTime': {'time': {'text': '3:38\u202fPM'},
          'timeZone': 'America/New_York'}},
        'headsign': 'South Ferry',
        'headway': '300s',
        'transitLine': {'agencies': [{'name': 'MTA New York City Transit',
           'phoneNumber': '+1 718-330-1234',
           'uri': 'http://www.mta.info/'}],
         'name': '1 Train (Broadway-7 Av Local)',
  

In [214]:
#Run api with test addresses
import requests

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Origin': 'https://www.mta.info',
    'Referer': 'https://www.mta.info/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

params = {
    'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
    'allowUnknownTransfers': 'false',
    'mode': 'TRANSIT,WALK',
    'numItineraries': '3',
    'showIntermediateStops': 'true',
    'smartKissAndRide': 'true',
    'flagStopBufferSize': '50',
    'nextDepartureWindow': '86400',
    'toPlace': '40.7424509, -74.0059581',
    'fromPlace': '40.8366459, -73.9471301',
    'arriveBy': 'false',
    'wheelchair': 'false',
    'maxWalkDistance': '804',
    'date': '7/29/25',
    'time': '03:55 PM',
    'optimize': 'QUICK',
    'bannedAgencies': '',
    'bannedRouteTypes': '',
}

response = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)

In [215]:
data = response.json()

In [220]:
data['plan']['itineraries'][1]['duration']

2658

In [207]:
A_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address_full  10 non-null     object 
 1   number        10 non-null     object 
 2   street        10 non-null     object 
 3   city          10 non-null     object 
 4   state         10 non-null     object 
 5   zip           10 non-null     int64  
 6   longitude     10 non-null     float64
 7   latitude      10 non-null     float64
 8   statefp       10 non-null     int64  
 9   countyfp      10 non-null     int64  
 10  tractce       10 non-null     int64  
 11  blkgrpce      10 non-null     int64  
 12  coordinates   10 non-null     object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.1+ KB


In [239]:
#add the coordinates to the dataframe as a single value
A_addresses['coordinates'] = A_addresses["latitude"].astype(str) + ", " + A_addresses["longitude"].astype(str)
A_addresses

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce,coordinates
0,"133-20 85 ST NEW YORK, NY 11417",133-20,85 ST,NEW YORK,NY,11417,-73.851338,40.673495,36,81,5800,3,"40.673495, -73.851338"
1,"292 MONTAUK AVE NEW YORK, NY 11208",292,MONTAUK AVE,NEW YORK,NY,11208,-73.877478,40.671479,36,47,119400,3,"40.671479, -73.877478"
2,"6 ROMER RD NEW YORK, NY 10304",6,ROMER RD,NEW YORK,NY,10304,-74.103098,40.593125,36,85,17701,3,"40.593125, -74.103098"
3,"164-10 104 ST NEW YORK, NY 11414",164-10,104 ST,NEW YORK,NY,11414,-73.827662,40.651995,36,81,88400,6,"40.651995, -73.827662"
4,"1429 SHORE PKWY NEW YORK, NY 11214",1429,SHORE PKWY,NEW YORK,NY,11214,-74.005699,40.598670,36,47,28200,1,"40.59867, -74.005699"
5,"196 22 ST NEW YORK, NY 11232",196,22 ST,NEW YORK,NY,11232,-73.995537,40.661591,36,47,14500,2,"40.661591, -73.995537"
6,"715 SAGAMORE ST NEW YORK, NY 10462",715,SAGAMORE ST,NEW YORK,NY,10462,-73.866310,40.848674,36,5,23000,1,"40.848674, -73.86631"
7,"4730 RICHARDSON AVE NEW YORK, NY 10470",4730,RICHARDSON AVE,NEW YORK,NY,10470,-73.850599,40.904675,36,5,41400,2,"40.904675, -73.850599"
8,"94-17 84 ST NEW YORK, NY 11416",94-17,84 ST,NEW YORK,NY,11416,-73.856868,40.685023,36,81,3400,1,"40.685023, -73.856868"
9,"2732 MILL AVE NEW YORK, NY 11234",2732,MILL AVE,NEW YORK,NY,11234,-73.914308,40.607894,36,47,70201,5,"40.607894, -73.914308"


In [240]:
#add coordinates as column
B_addresses['coordinates'] = B_addresses["latitude"].astype(str) + ", " + B_addresses["longitude"].astype(str)
B_addresses

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce,coordinates
0,"3236 BOSTON RD NEW YORK, NY 10469",3236,BOSTON RD,NEW YORK,NY,10469,-73.854792,40.872162,36,5,34800,1,"40.872162, -73.854792"
1,"970 AMSTERDAM AVE NEW YORK, NY 10025",970,AMSTERDAM AVE,NEW YORK,NY,10025,-73.965107,40.801581,36,61,19500,4,"40.801581, -73.965107"
2,"1623 DECATUR ST NEW YORK, NY 11385",1623,DECATUR ST,NEW YORK,NY,11385,-73.901360,40.694898,36,81,55900,1,"40.694898, -73.90136"
3,"12-06 151 PL NEW YORK, NY 11357",12-06,151 PL,NEW YORK,NY,11357,-73.811475,40.791270,36,81,98700,2,"40.79127, -73.811475"
4,"135 BCH 56 PL NEW YORK, NY 11692",135,BCH 56 PL,NEW YORK,NY,11692,-73.787031,40.590884,36,81,97202,1,"40.590884, -73.787031"
5,"196 ROSEDALE AVE NEW YORK, NY 10312",196,ROSEDALE AVE,NEW YORK,NY,10312,-74.195661,40.547968,36,85,17014,1,"40.547968, -74.195661"
6,"519 MADISON AVE NEW YORK, NY 10022",519,MADISON AVE,NEW YORK,NY,10022,-73.973895,40.759931,36,61,10200,1,"40.759931, -73.973895"
7,"197-26 CARPENTER AVE NEW YORK, NY 11423",197-26,CARPENTER AVE,NEW YORK,NY,11423,-73.762244,40.711682,36,81,50000,1,"40.711682, -73.762244"
8,"63-62 77 ST NEW YORK, NY 11379",63-62,77 ST,NEW YORK,NY,11379,-73.878306,40.718089,36,81,66100,1,"40.718089, -73.878306"
9,"120-36 143 ST NEW YORK, NY 11436",120-36,143 ST,NEW YORK,NY,11436,-73.795799,40.675556,36,81,18200,1,"40.675556, -73.795799"


In [241]:
#make that column a list
A_coors = A_addresses['coordinates'].tolist()
A_coors

['40.673495, -73.851338',
 '40.671479, -73.877478',
 '40.593125, -74.103098',
 '40.651995, -73.827662',
 '40.59867, -74.005699',
 '40.661591, -73.995537',
 '40.848674, -73.86631',
 '40.904675, -73.850599',
 '40.685023, -73.856868',
 '40.607894, -73.914308']

In [242]:
#make that column a list
B_coors = B_addresses['coordinates'].tolist()
B_coors

['40.872162, -73.854792',
 '40.801581, -73.965107',
 '40.694898, -73.90136',
 '40.79127, -73.811475',
 '40.590884, -73.787031',
 '40.547968, -74.195661',
 '40.759931, -73.973895',
 '40.711682, -73.762244',
 '40.718089, -73.878306',
 '40.675556, -73.795799']

In [253]:
#Run all addresses for a regular commute
durations=[]
for A_loc, B_loc in zip(A_coors, B_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': B_loc,
        'fromPlace': A_loc,
        'arriveBy': 'false',
        'wheelchair': 'false',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': '',
        'bannedRouteTypes': '',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }
    response = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    data = response.json()
    print(data['plan']['itineraries'][0]['duration'])
    durations.append(data['plan']['itineraries'][0]['duration'])
        

6564
4537
8447
6303
6736
6321
3078
6839
2178
7519


In [256]:
durations

[6564, 4537, 8447, 6303, 6736, 6321, 3078, 6839, 2178, 7519]

In [270]:
#Run for an accessible commute
durations_accessible=[]
for A_loc, B_loc in zip(A_coors, B_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': B_loc,
        'fromPlace': A_loc,
        'arriveBy': 'false',
        'wheelchair': 'true',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': '',
        'bannedRouteTypes': '',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }
    response_accessible = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    data_accessible = response_accessible.json()
    print(data_accessible['plan']['itineraries'][0]['duration'])
    durations_accessible.append(data_accessible['plan']['itineraries'][0]['duration'])

8831
6305
8447
6303
6736
8101
3419
9356
2178
7519


In [271]:
durations_accessible

[8831, 6305, 8447, 6303, 6736, 8101, 3419, 9356, 2178, 7519]

In [272]:
#save the values to a dataframe
import pandas as pd

df = pd.DataFrame({'commute_time' : durations})
df['accessible_commute_time'] = pd.DataFrame(durations_accessible)
df

,commute_time,accessible_commute_time
0,6564,8831
1,4537,6305
2,8447,8447
3,6303,6303
4,6736,6736
5,6321,8101
6,3078,3419
7,6839,9356
8,2178,2178
9,7519,7519


In [273]:
#create a column to show the difference in commute time
df['time_difference'] = df['accessible_commute_time'] - df['commute_time']
df

,commute_time,accessible_commute_time,time_difference
0,6564,8831,2267
1,4537,6305,1768
2,8447,8447,0
3,6303,6303,0
4,6736,6736,0
5,6321,8101,1780
6,3078,3419,341
7,6839,9356,2517
8,2178,2178,0
9,7519,7519,0


In [275]:
#Let's get a larger data set to do a better analysis
origin_addresses = pd.read_csv('origin_addresses.csv')
destination_addresses = pd.read_csv('destination_addresses.csv')

In [276]:
origin_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"150 E 57 ST NEW YORK, NY 10022",150,E 57 ST,NEW YORK,NY,10022,-73.968580,40.760489,36,61,11203,1
1,"301 PARK PL NEW YORK, NY 11238",301,PARK PL,NEW YORK,NY,11238,-73.967773,40.676629,36,47,20500,3
2,"206 COTTER AVE NEW YORK, NY 10306",206,COTTER AVE,NEW YORK,NY,10306,-74.142019,40.564129,36,85,14606,4
3,"37 VAN CORTLANDT AVE NEW YORK, NY 10301",37,VAN CORTLANDT AVE,NEW YORK,NY,10301,-74.099542,40.616620,36,85,4700,1
4,"315 E 58 ST NEW YORK, NY 10022",315,E 58 ST,NEW YORK,NY,10022,-73.964063,40.759789,36,61,10802,2


In [277]:
destination_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce
0,"125 MEISNER AVE NEW YORK, NY 10306",125,MEISNER AVE,NEW YORK,NY,10306,-74.131756,40.579843,36,85,27900,1
1,"31 W 71 ST NEW YORK, NY 10023",31,W 71 ST,NEW YORK,NY,10023,-73.977690,40.776165,36,61,15700,3
2,"660 BAY ST NEW YORK, NY 10304",660,BAY ST,NEW YORK,NY,10304,-74.075304,40.625884,36,85,2700,2
3,"318 MOSEL AVE NEW YORK, NY 10304",318,MOSEL AVE,NEW YORK,NY,10304,-74.080679,40.608834,36,85,4004,1
4,"303 ARLINGTON AVE NEW YORK, NY 11208",303,ARLINGTON AVE,NEW YORK,NY,11208,-73.883413,40.681170,36,47,117202,2


In [278]:
#add the combined coordinates to the dataframe as a single column
origin_addresses['coordinates'] = origin_addresses["latitude"].astype(str) + ", " + origin_addresses["longitude"].astype(str)
origin_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce,coordinates
0,"150 E 57 ST NEW YORK, NY 10022",150,E 57 ST,NEW YORK,NY,10022,-73.968580,40.760489,36,61,11203,1,"40.760489, -73.96858"
1,"301 PARK PL NEW YORK, NY 11238",301,PARK PL,NEW YORK,NY,11238,-73.967773,40.676629,36,47,20500,3,"40.676629, -73.967773"
2,"206 COTTER AVE NEW YORK, NY 10306",206,COTTER AVE,NEW YORK,NY,10306,-74.142019,40.564129,36,85,14606,4,"40.564129, -74.142019"
3,"37 VAN CORTLANDT AVE NEW YORK, NY 10301",37,VAN CORTLANDT AVE,NEW YORK,NY,10301,-74.099542,40.616620,36,85,4700,1,"40.61662, -74.099542"
4,"315 E 58 ST NEW YORK, NY 10022",315,E 58 ST,NEW YORK,NY,10022,-73.964063,40.759789,36,61,10802,2,"40.759789, -73.964063"


In [279]:
destination_addresses['coordinates'] = destination_addresses["latitude"].astype(str) + ", " + destination_addresses["longitude"].astype(str)
destination_addresses.head()

,address_full,number,street,city,state,zip,longitude,latitude,statefp,countyfp,tractce,blkgrpce,coordinates
0,"125 MEISNER AVE NEW YORK, NY 10306",125,MEISNER AVE,NEW YORK,NY,10306,-74.131756,40.579843,36,85,27900,1,"40.579843, -74.131756"
1,"31 W 71 ST NEW YORK, NY 10023",31,W 71 ST,NEW YORK,NY,10023,-73.977690,40.776165,36,61,15700,3,"40.776165, -73.97769"
2,"660 BAY ST NEW YORK, NY 10304",660,BAY ST,NEW YORK,NY,10304,-74.075304,40.625884,36,85,2700,2,"40.625884, -74.075304"
3,"318 MOSEL AVE NEW YORK, NY 10304",318,MOSEL AVE,NEW YORK,NY,10304,-74.080679,40.608834,36,85,4004,1,"40.608834, -74.080679"
4,"303 ARLINGTON AVE NEW YORK, NY 11208",303,ARLINGTON AVE,NEW YORK,NY,11208,-73.883413,40.681170,36,47,117202,2,"40.68117, -73.883413"


In [281]:
#make that column a list
origin_coors = origin_addresses['coordinates'].tolist()
len(origin_coors)

destination_coors = destination_addresses['coordinates'].tolist()
len(destination_coors)

100

In [293]:
#Run all addresses for a regular commute
import numpy as np

full_durations_regular=[]
for origin_loc, destination_loc in zip(origin_coors, destination_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': destination_loc,
        'fromPlace': origin_loc,
        'arriveBy': 'false',
        'wheelchair': 'false',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': '',
        'bannedRouteTypes': '',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }

    response_regular = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    full_data_regular = response_regular.json()
    try:
        print(full_data_regular['plan']['itineraries'][0]['duration'])
        full_durations_regular.append(full_data_regular['plan']['itineraries'][0]['duration'])
    except:
        print("no route")
        full_durations_regular.append(np.nan)

5925
2458
2855
1402
3694
2559
4698
4017
3284
3473
3922
2937
5351
2982
4862
3405
5635
1784
6520
2607
12287
3980
7909
4679
2762
6915
4947
8000
2128
4685
3697
3067
6117
5345
8606
4470
8470
10105
6885
1686
7024
3015
8861
8177
5199
5170
1569
8627
2860
2856
7448
3496
9020
7674
7936
9120
6910
3999
10865
1256
9188
5736
4412
3745
7443
8636
8745
no route
5407
8735
2807
9333
8526
4387
1954
11192
7645
8211
5364
3892
6570
9204
4487
6709
1578
4639
3815
4147
2849
6015
4561
3016
7186
8502
5349
3406
3860
5997
no route
3983


In [294]:
len(full_durations_regular)

100

In [295]:
#Run them all for the accessible commute
full_durations_accessible=[]
for origin_loc, destination_loc in zip(origin_coors, destination_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': destination_loc,
        'fromPlace': origin_loc,
        'arriveBy': 'false',
        'wheelchair': 'true',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': '',
        'bannedRouteTypes': '',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }
    response_accessible = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    full_data_accessible = response_accessible.json()
    try:
        print(full_data_accessible['plan']['itineraries'][0]['duration'])
        full_durations_accessible.append(full_data_accessible['plan']['itineraries'][0]['duration'])
    except:
        print("no route")
        full_durations_accessible.append(np.nan)

5925
2992
3387
1402
4155
3668
4698
4017
3284
3320
3922
2937
9933
2982
7495
4940
8669
1784
6764
3357
12287
5698
7909
5502
3680
6915
5881
8000
2702
4685
3697
4161
6117
7623
8606
5011
8470
10105
9946
1686
7684
3787
8861
8177
5199
5170
1430
8627
3000
3123
7448
3812
9020
8935
7936
10331
6910
4776
10865
1256
9188
5736
4412
3745
7774
8636
10278
no route
5960
8735
3873
9036
9297
4387
2770
12729
10133
8211
5364
4324
7350
9204
8302
6709
2273
4639
3815
6240
2849
6419
6390
3016
7186
8502
5491
4887
3973
6499
no route
5966


In [296]:
len(full_durations_accessible)

100

In [299]:
#Now, I will construct a useful dataframe

df_full = pd.DataFrame({'commute_time' : full_durations_regular})
df_full['accessible_commute_time'] = pd.DataFrame(full_durations_accessible)
#create a column to show the difference in commute time
df_full['time_difference'] = df_full['accessible_commute_time'] - df_full['commute_time']
df_full.head()

,commute_time,accessible_commute_time,time_difference
0,5925.0,5925.0,0.0
1,2458.0,2992.0,534.0
2,2855.0,3387.0,532.0
3,1402.0,1402.0,0.0
4,3694.0,4155.0,461.0


In [300]:
df_full['origin_address'] = origin_addresses['address_full']
df_full['destination_address'] = destination_addresses['address_full']
df_full['origin_coor'] = origin_addresses['coordinates']
df_full['desinaiton_coor'] = destination_addresses['coordinates']
df_full.head()

,commute_time,accessible_commute_time,time_difference,origin_address,destination_address,origin_coor,desinaiton_coor
0,5925.0,5925.0,0.0,"150 E 57 ST NEW YORK, NY 10022","125 MEISNER AVE NEW YORK, NY 10306","40.760489, -73.96858","40.579843, -74.131756"
1,2458.0,2992.0,534.0,"301 PARK PL NEW YORK, NY 11238","31 W 71 ST NEW YORK, NY 10023","40.676629, -73.967773","40.776165, -73.97769"
2,2855.0,3387.0,532.0,"206 COTTER AVE NEW YORK, NY 10306","660 BAY ST NEW YORK, NY 10304","40.564129, -74.142019","40.625884, -74.075304"
3,1402.0,1402.0,0.0,"37 VAN CORTLANDT AVE NEW YORK, NY 10301","318 MOSEL AVE NEW YORK, NY 10304","40.61662, -74.099542","40.608834, -74.080679"
4,3694.0,4155.0,461.0,"315 E 58 ST NEW YORK, NY 10022","303 ARLINGTON AVE NEW YORK, NY 11208","40.759789, -73.964063","40.68117, -73.883413"


In [302]:
#on average, how much longer are the accessible commutes? 
df_full['time_difference'].mean()/60

np.float64(9.444557823129252)

In [304]:
#9 minutes isn't a lot. But the ones that have a time difference of 0 are commutes that are between two accessible stations. 
#On average, how much longer is a commute that has to adjust to accomodate a non-accessible station? 

df_full['time_diff_edit'] = df_full['time_difference'].replace(0, np.nan)


In [305]:
df_full['time_diff_edit'].mean()/60

np.float64(18.511333333333333)

In [308]:
#18.5 min...still not as long as i'd have thought, but considerably longer
#What was the longest time difference? 
df_full['time_difference'].max()/60

np.float64(76.36666666666666)

In [310]:
df_sorted = df_full.sort_values(by='time_difference', ascending = False)
df_sorted.head()

,commute_time,accessible_commute_time,time_difference,origin_address,destination_address,origin_coor,desinaiton_coor,time_diff_edit
12,5351.0,9933.0,4582.0,"738 41 ST NEW YORK, NY 11232","31-37 76 ST NEW YORK, NY 11370","40.646654, -73.999919","40.757557, -73.891232",4582.0
82,4487.0,8302.0,3815.0,"535 W 155 ST NEW YORK, NY 10032","1060 OCEAN AVE NEW YORK, NY 11226","40.832531, -73.944865","40.636546, -73.958645",3815.0
38,6885.0,9946.0,3061.0,"2014 52 ST NEW YORK, NY 11204","201-12 EPSOM CRSE NEW YORK, NY 11427","40.623533, -73.977885","40.72479, -73.764218",3061.0
16,5635.0,8669.0,3034.0,"20 HAMILTON TER NEW YORK, NY 10031","11 BLEEKER PL NEW YORK, NY 10314","40.822503, -73.946849","40.601229, -74.163802",3034.0
14,4862.0,7495.0,2633.0,"1660 51 ST NEW YORK, NY 11204","45-41 47 ST NEW YORK, NY 11377","40.628888, -73.985233","40.741671, -73.917918",2633.0


In [311]:
#Did everything that had a regular route, also offer an accessible route? 
df_full['commute_time'].isna().sum()

np.int64(2)

In [312]:
df_full['accessible_commute_time'].isna().sum()

np.int64(2)

In [315]:
#Yes, there was an accessible alternative for every commute that had a regular route
#What if we exclude bus routes? Then how much longer is the accessible commute? 
full_durations_no_bus=[]
for origin_loc, destination_loc in zip(origin_coors, destination_coors):
    params = {
        'apikey': 'Z276E3rCeTzOQEoBPPN4JCEc6GfvdnYE',
        'allowUnknownTransfers': 'false',
        'mode': 'TRANSIT,WALK',
        'numItineraries': '3',
        'showIntermediateStops': 'true',
        'smartKissAndRide': 'true',
        'flagStopBufferSize': '50',
        'nextDepartureWindow': '86400',
        'toPlace': destination_loc,
        'fromPlace': origin_loc,
        'arriveBy': 'false',
        'wheelchair': 'true',
        'maxWalkDistance': '804',
        'date': '7/29/25',
        'time': '03:55 PM',
        'optimize': 'QUICK',
        'bannedAgencies': ' MTABC, MTA NYCT',
        'bannedRouteTypes': '3,702',
            }
    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Origin': 'https://www.mta.info',
        'Referer': 'https://www.mta.info/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        }
    response_no_bus = requests.get('https://otp-mta-prod.camsys-apps.com/otp/routers/default/plan', params=params, headers=headers)
    full_data_no_bus = response_no_bus.json()
    try:
        print(full_data_no_bus['plan']['itineraries'][0]['duration'])
        full_durations_no_bus.append(full_data_no_bus['plan']['itineraries'][0]['duration'])
    except:
        print("no route")
        full_durations_no_bus.append(np.nan)
    

no route
no route
no route
2112
no route
5794
no route
no route
5322
no route
5974
5123
no route
5393
no route
no route
no route
2706
no route
3927
no route
no route
no route
no route
5838
no route
no route
no route
5878
11543
no route
no route
no route
no route
no route
no route
no route
no route
no route
3665
no route
no route
no route
no route
no route
no route
2768
no route
3248
no route
no route
no route
no route
no route
no route
no route
no route
no route
no route
805
no route
no route
no route
no route
no route
no route
no route
no route
no route
no route
4603
no route
no route
no route
6102
no route
no route
no route
no route
no route
no route
no route
no route
no route
2334
no route
no route
no route
4494
no route
no route
no route
no route
no route
9068
no route
no route
no route
no route
no route


In [316]:
df_full['accessible_no_bus'] = pd.DataFrame(full_durations_no_bus)
df_full['no_bus_time_difference'] = df_full['accessible_no_bus'] - df_full['commute_time']
df_full.head()

,commute_time,accessible_commute_time,time_difference,origin_address,destination_address,origin_coor,desinaiton_coor,time_diff_edit,accessible_no_bus,no_bus_time_difference
0,5925.0,5925.0,0.0,"150 E 57 ST NEW YORK, NY 10022","125 MEISNER AVE NEW YORK, NY 10306","40.760489, -73.96858","40.579843, -74.131756",NaN,NaN,NaN
1,2458.0,2992.0,534.0,"301 PARK PL NEW YORK, NY 11238","31 W 71 ST NEW YORK, NY 10023","40.676629, -73.967773","40.776165, -73.97769",534.0,NaN,NaN
2,2855.0,3387.0,532.0,"206 COTTER AVE NEW YORK, NY 10306","660 BAY ST NEW YORK, NY 10304","40.564129, -74.142019","40.625884, -74.075304",532.0,NaN,NaN
3,1402.0,1402.0,0.0,"37 VAN CORTLANDT AVE NEW YORK, NY 10301","318 MOSEL AVE NEW YORK, NY 10304","40.61662, -74.099542","40.608834, -74.080679",NaN,2112.0,710.0
4,3694.0,4155.0,461.0,"315 E 58 ST NEW YORK, NY 10022","303 ARLINGTON AVE NEW YORK, NY 11208","40.759789, -73.964063","40.68117, -73.883413",461.0,NaN,NaN


In [318]:
df_full['no_bus_time_difference'].mean()/60

np.float64(36.4475)

In [319]:
#Ok, I don't think the MTA does a very good job of accounting for traffic in the bus times. I'll try to do a sample of that with the google api
import os
from dotenv import load_dotenv
load_dotenv() 
API_KEY = os.getenv("api_key")

In [335]:
import requests
import json

headers = {
    'content-type': 'application/json; application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
}

data = '{\n  "origin": {\n    "address": "853 Riverside Dr, New York, NYn  },\n  "destination": {\n    "address": "75 9th Ave, New York, NY"\n  },\n  "travelMode": "TRANSIT",\n  "computeAlternativeRoutes": true,\n  "transitPreferences": {\n     routingPreference: "LESS_WALKING",\n     allowedTravelModes: ["TRAIN"]\n  },\n}'.encode()

response = requests.post('https://routes.googleapis.com/directions/v2:computeRoutes', headers=headers, data=data)


In [337]:
response

<Response [400]>